<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras import layers
from google.colab import files
!pip install talos
import talos

In [359]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [360]:
train_df.shape

(7613, 2)

In [361]:
x_train, x_test, y_train, y_test = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [362]:
#vectorizer = CountVectorizer()
#vectorizer.fit(x_train)

#x_train = vectorizer.transform(x_train)
#x_test = vectorizer.transform(x_test)
#x_train

In [363]:
#input_dim = x_train.shape[1]

#model = Sequential()
#model.add(layers.Dense(4,input_dim=input_dim,activation='relu'))
#model.add(layers.Dense(1,activation = 'sigmoid'))

In [364]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

In [365]:
#history = model.fit(x_train,y_train,epochs = 10,verbose = 1,validation_data = (x_test,y_test),batch_size=15)

In [366]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model

In [367]:
p = {
    'first neuron units': (2,20,1),
    'first neuron activation': ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'epochs': [1,2,3,4,5,6,7,10,20,100],
    'batch size': (5,100,5)
}

In [368]:
#loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

In [369]:
x_train.shape
t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

100%|██████████| 300/300 [2:36:21<00:00, 31.27s/it] 


In [370]:
print('test')

test


In [371]:
experiment_data = t.data
experiment_data.head()

,round_epochs,val_loss,val_accuracy,loss,accuracy,batch size,epochs,first neuron activation,first neuron units
0,1,0.533879,0.794658,0.606395,0.730343,5,1,relu,2
1,1,0.471098,0.799912,0.550330,0.757553,5,1,elu,2
2,1,0.622869,0.626532,0.660108,0.577031,5,1,sigmoid,2
3,1,0.569834,0.775394,0.621339,0.694502,5,1,softmax,2
4,1,0.485486,0.795096,0.569680,0.748358,5,1,tanh,2


In [372]:
experiment_data['val_accuracy'].idxmax()

26

In [373]:
experiment_data.iloc[205]

round_epochs                      5
val_loss                   0.491018
val_accuracy               0.795534
loss                       0.409134
accuracy                   0.882154
batch size                       62
epochs                            5
first neuron activation         elu
first neuron units                2
Name: 205, dtype: object

In [378]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])

data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [379]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_test, y_train, y_test = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)


input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Dense(2,input_dim=input_dim,activation='elu'))
best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(x_train,y_train,epochs = 5,verbose = 1,validation_data = (x_test,y_test),batch_size=62)

loss, accuracy = best_model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 2s 287us/step - loss: 0.6355 - accuracy: 0.7246 - val_loss: 0.5852 - val_accuracy: 0.7610
Epoch 2/5
6090/6090 [==============================] - 2s 264us/step - loss: 0.5194 - accuracy: 0.8258 - val_loss: 0.5256 - val_accuracy: 0.7938
Epoch 3/5
6090/6090 [==============================] - 2s 268us/step - loss: 0.4365 - accuracy: 0.8640 - val_loss: 0.4899 - val_accuracy: 0.8030
Epoch 4/5
6090/6090 [==============================] - 2s 267us/step - loss: 0.3719 - accuracy: 0.8883 - val_loss: 0.4690 - val_accuracy: 0.8122
Epoch 5/5
6090/6090 [==============================] - 2s 265us/step - loss: 0.3208 - accuracy: 0.9066 - val_loss: 0.4559 - val_accuracy: 0.8168
Training Accuracy: 0.9223
Testing Accuracy:  0.8168


Miren lo buenos que son los resultados con UNA SOLA CAPA DENSE.
Sin embedding nada.
Imaginense los resultados con capas convolucionales y un buen embedding. Eso es lo que les invito a añadir a uds que estuvieron practicando con eso. Yo tuve muchos problemas para hacer un submit, porque no entiendo bien como funciona el CountVectorizer. Voy a seguir con mecanismos de gradient boosting ahora. Por favor hagamos tuning de hiperparámetros a todos los modelos que usemos para poder exprimirlos bien. Sino tenemos una version muy light de los modelos.

In [380]:
#Reentrena con el set completo y realiza predicción.

out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
data_test = vectorizer.transform(test_df['text']).toarray()
prediction=best_model.predict_classes(data_test, batch_size=512)
prediction

Epoch 1/5
7613/7613 [==============================] - 2s 241us/step - loss: 0.3125 - accuracy: 0.8993
Epoch 2/5
7613/7613 [==============================] - 2s 233us/step - loss: 0.2664 - accuracy: 0.9176
Epoch 3/5
7613/7613 [==============================] - 2s 242us/step - loss: 0.2305 - accuracy: 0.9345
Epoch 4/5
7613/7613 [==============================] - 2s 236us/step - loss: 0.2012 - accuracy: 0.9443
Epoch 5/5
7613/7613 [==============================] - 2s 236us/step - loss: 0.1766 - accuracy: 0.9535


array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [381]:
# Genera archivo submit.

test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')